<a href="https://colab.research.google.com/github/patrickhamzaokello/Recommender_System_Mx/blob/main/Recommendation_System_(official).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install librosa
!pip install mysql-connector-python
!pip install mysql-connector-python pandas scikit-learn tensorflow
!pip install pymysql pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 28.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.21.12 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 968.4 kB/s eta 0:00:00


In [2]:
import pymysql

import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import librosa
import requests
import io
import json

from datetime import date

In [3]:
# Connect to the MySQL database
db = pymysql.connect(
    host="178.79.148.46",
    user="Colab",
    password="123456789m",
    database="mwonya"
)

# Create a cursor to interact with the database
cursor = db.cursor()

In [4]:
# Retrieve song table
cursor.execute("SELECT  f.userid as user_id,f.songid as song_id,f.plays as total_plays, f.lastPlayed as time_played FROM frequency f  INNER JOIN songs s on f.songid = s.id")
columns = cursor.description
result_set = [{columns[index][0]: column for index, column in enumerate(value)} for value in cursor.fetchall()]
ratings_df=pd.DataFrame(result_set)
ratings_df.shape
ratings_df.head()

,user_id,song_id,total_plays,time_played
0,mw6375a9ee98402kin,74,1,2022-11-28 17:31:21
1,mw613dd8249ec3cLam,74,1,2023-02-27 15:35:21
2,mwgwSU9qf9S3cymVCT4CgUBV3xkXf2,74,1,2023-03-07 09:39:53
3,mwZy28HEiwVDWh6zMvii8iY0F5dDz1,74,1,2023-06-14 22:08:49
4,mw64ee30f47c5c3eba1693331700,74,1,2023-09-29 10:07:51


In [5]:
# Retrieve song table
cursor.execute("SELECT s.id as song_id, s.title as song_name, g.name as song_genre FROM songs s join genres g on s.genre=g.id")
columns = cursor.description
result_set = [{columns[index][0]: column for index, column in enumerate(value)} for value in cursor.fetchall()]
songs_df=pd.DataFrame(result_set)
songs_df.shape
songs_df.head()

,song_id,song_name,song_genre
0,74,00 Waidaba New Season Intro,Audio Dramas
1,75,01 Waidaba episode industry,Audio Dramas
2,76,02 Waidaba Episode Purpose,Audio Dramas
3,77,Peloke,Afro pop
4,78,One more night,Afro pop


In [6]:
# Retrieve user table
cursor.execute("SELECT id as user_id, firstName as full_name, signUpDate as date_joined FROM users")
columns = cursor.description
result_set = [{columns[index][0]: column for index, column in enumerate(value)} for value in cursor.fetchall()]
df=pd.DataFrame(result_set)
df.shape
df.head()

,user_id,full_name,date_joined
0,mw603382d49906aPka,Patrick,2021-02-22
1,mw603540ce5fbc8fai,Mugi,2021-02-23
2,mw604111cfe8676Obo,Denis,2021-03-04
3,mw604111d077103Ton,Toni,2021-03-04
4,mw6041ff60380e7jim,Jimmy,2021-03-05


In [7]:
#map songs id to tile
song_names = songs_df.set_index('song_id')['song_name'].to_dict()
# map user id to name
user_names = df.set_index('user_id')['full_name'].to_dict()

n_users = len(ratings_df.user_id.unique())
n_items = len(ratings_df.song_id.unique())

print("number of unique users: ",n_users)
print("number of unique songs: ", n_items)
print("The full rating matrix will have: ", n_users * n_items, "elements")

print("----")
print("Number of ratings:",len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users * n_items) *100, "% of the matrix is filled")
print("growth is by n*2")

number of unique users:  721
number of unique songs:  1296
The full rating matrix will have:  934416 elements
----
Number of ratings: 7829
Therefore:  0.8378495231246041 % of the matrix is filled
growth is by n*2


In [8]:
import torch
import numpy as np
from torch.autograd import variable
from tqdm import tqdm_notebook as tqdm

In [9]:
class MatrixFactorization(torch.nn.Module):
  def __init__(self, n_users, n_items, n_factors=20):
    super().__init__()
    #create user embeddings
    self.user_factors = torch.nn.Embedding(n_users, n_factors) # look up table for input
    #create item embeddings
    self.item_factors = torch.nn.Embedding(n_items, n_factors) #think of this as look up table for input

    self.user_factors.weight.data.uniform_(0,0.05)
    self.item_factors.weight.data.uniform_(0,0.05)

  def forward(self, data):
    #matrix multiplication
    users, items = data[:,0], data[:,1]
    return (self.user_factors(users)*self.item_factors(items)).sum(1)

  def predict(self, user, item):
    return self.forward(user, item)


In [10]:
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

class Loader(Dataset):
  def __init__(self):
    self.ratings = ratings_df.copy()

    #extract all user ids and song ids
    users = ratings_df.user_id.unique()
    songs = ratings_df.song_id.unique()

    #Produce new continuous IDs for users and songs
    self.userid2idx = {o:i for i,o in enumerate(users)}
    self.songid2idx = {o:i for i,o in enumerate(songs)}

    #obtained continuouse ID for users and songs
    self.idx2userid = {i:o for o,i in self.userid2idx.items()}
    self.idx2songid = {i:o for o,i in self.songid2idx.items()}

    #return the id from the indexed values as noted in the lambda function down below
    self.ratings.song_id = ratings_df.song_id.apply(lambda x: self.songid2idx[x])
    self.ratings.user_id = ratings_df.user_id.apply(lambda x: self.userid2idx[x])


    self.x = self.ratings.drop(['total_plays','time_played'], axis=1).values
    self.y = self.ratings['total_plays'].values
    self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # transforms the data into tensors (ready for torch models)

  def __getitem__(self, index):
    return (self.x[index], self.y[index])

  def __len__(self):
    return len(self.ratings)



In [13]:
num_epochs = 1000
cuda = torch.cuda.is_available()

print("Is Running on GPU: ", cuda)

model = MatrixFactorization(n_users, n_items,n_factors=8)
print(model)


for name, param in model.named_parameters():
  if param.requires_grad:
    print(name, param.data)

#GPU enabl if you a have a GPU
if cuda:
  model = model.cuda()

#MSE loss
loss_fn = torch.nn.MSELoss()

#Adam optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

train_set = Loader()
train_loader = DataLoader(train_set, 128,shuffle=True)

Is Running on GPU:  False
MatrixFactorization(
  (user_factors): Embedding(721, 8)
  (item_factors): Embedding(1296, 8)
)
user_factors.weight tensor([[0.0168, 0.0239, 0.0345,  ..., 0.0182, 0.0010, 0.0363],
        [0.0242, 0.0367, 0.0345,  ..., 0.0142, 0.0244, 0.0423],
        [0.0349, 0.0401, 0.0027,  ..., 0.0264, 0.0230, 0.0119],
        ...,
        [0.0130, 0.0173, 0.0431,  ..., 0.0157, 0.0275, 0.0261],
        [0.0473, 0.0108, 0.0107,  ..., 0.0148, 0.0282, 0.0104],
        [0.0396, 0.0048, 0.0313,  ..., 0.0383, 0.0104, 0.0230]])
item_factors.weight tensor([[0.0204, 0.0068, 0.0254,  ..., 0.0392, 0.0003, 0.0411],
        [0.0078, 0.0241, 0.0437,  ..., 0.0124, 0.0468, 0.0190],
        [0.0355, 0.0183, 0.0444,  ..., 0.0459, 0.0322, 0.0172],
        ...,
        [0.0085, 0.0267, 0.0333,  ..., 0.0237, 0.0023, 0.0183],
        [0.0284, 0.0002, 0.0060,  ..., 0.0332, 0.0051, 0.0278],
        [0.0013, 0.0457, 0.0167,  ..., 0.0042, 0.0265, 0.0348]])


In [ ]:
# train the model
for it in tqdm(range(num_epochs)):
  losses = []
  for x,y in train_loader:
    if cuda:
      x,y = x.cuda(), y.cuda()
      optimizer.zero_grad()
      outputs = model(x)
      loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
      losses.append(loss.item())
      loss.backward()
      optimizer.step()
  print("iter #{}".format(it), "Loss:", sum(losses)/ len(losses))

In [ ]:
# by training the model, we will have tuned laten factors for songs and users
c = 0
uw = 0
iw = 0

for name,param in model.named_parameters():
  if param.requires_grad:
    print(name, param.data)
    if c == 0:
      uw = param.data
      c += 1
    else:
      iw = param.data

user_factors.weight tensor([[ 6.2581,  8.0405,  8.8356,  ...,  9.5124,  7.6583,  2.3695],
        [ 0.3114,  2.0481, -1.3864,  ...,  0.9993,  0.1714,  0.8606],
        [ 0.6733,  0.2134,  0.2031,  ...,  0.1710, -0.3790,  0.7404],
        ...,
        [ 0.6967,  0.7067,  0.6206,  ...,  0.5934,  0.3144,  0.7316],
        [ 1.1105,  0.5645,  0.7894,  ...,  1.2721,  0.6719,  0.7300],
        [ 0.5158,  0.4479,  0.4929,  ...,  0.5162,  0.4671,  0.5081]],
       device='cuda:0')
item_factors.weight tensor([[ 0.4051, -0.0904,  0.1167,  ..., -0.0720, -0.3822,  0.8604],
        [-0.7063,  1.5292,  1.3422,  ...,  1.7109,  0.2560,  1.3188],
        [ 0.6886,  0.1792,  0.0607,  ..., -0.2052, -0.0787,  0.5456],
        ...,
        [ 0.3227,  0.1644,  0.1005,  ...,  0.0865,  0.2405,  0.1332],
        [ 0.9567,  0.9659,  0.7286,  ...,  0.9531,  0.9493,  0.4825],
        [ 0.2283,  0.2156,  0.1782,  ...,  0.2283,  0.2572,  0.1091]],
       device='cuda:0')


In [ ]:
trained_song_embeddings = model.item_factors.weight.data.cpu().numpy()

In [ ]:
len(trained_song_embeddings)  # unique song factor weights

1292

In [ ]:
trained_song_embeddings

array([[ 0.4050966 , -0.09035273,  0.11665162, ..., -0.07199065,
        -0.38217372,  0.8603617 ],
       [-0.7063016 ,  1.5291501 ,  1.3422145 , ...,  1.7109307 ,
         0.25601956,  1.3188343 ],
       [ 0.68859047,  0.17920983,  0.06071896, ..., -0.20517819,
        -0.07871293,  0.5456175 ],
       ...,
       [ 0.32265761,  0.16438188,  0.10048471, ...,  0.0864882 ,
         0.24054125,  0.13321514],
       [ 0.95674396,  0.9659433 ,  0.72863513, ...,  0.9530714 ,
         0.9492836 ,  0.48249894],
       [ 0.22828211,  0.21555555,  0.1782204 , ...,  0.22825286,
         0.25715294,  0.10910168]], dtype=float32)

In [ ]:
from sklearn.cluster import KMeans

#fit the clusters based on the song weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_song_embeddings)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
for cluster in range(10):
  print("Cluster #{}". format(cluster))
  tracks = []
  for trackidx in np.where(kmeans.labels_ == cluster)[0]:
    trackId = train_set.idx2songid[trackidx]
    rat_count = ratings_df.loc[ratings_df['song_id']==trackId].count()[0]
    tracks.append((song_names[trackId], rat_count))
    # print("song {}--{}".format(song_names[trackId],rat_count))

  for track in sorted(tracks, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", track[0],track[1])

Cluster #0
	 An ayele 47
	 Yahweh 38
	 Strength 34
	 Who Am I 32
	 Anoiting Flow 30
	 Episode 1 - Ugandan Teacher Selling Maize to Make Ends Meet 27
	 Tica 25
	 Miss You 19
	 Tana Ganj - Bebe X Dj Deo 19
	 Energy 18
Cluster #1
	 GEN IKOM RWOT - AKO MAR 17
	 PETYE GIN ALOI YESU BY CHRISTINE AKULLU 15
	 Words of Peace 11
	 Jesus you love me 7
Cluster #2
	 Akello 44
	 Ezimba  Dance along the beat 28
	 Beautifully Broken 28
	 Mummy 22
	 Sacrifice  22
	 Mr Kampala  18
	 Best of Afro Dance Music Session 01 17
	 Somamu_ Bob Narry 16
	 The Way You Prove It By Prof Jack Bell  16
	 Wakulila Wa 16
Cluster #3
	 Call on me(Nyeta) ft Agaba Banjo 36
	 Let the sky breathe ft Marili 24
	 Muniga 20
	 Amuge 17
	  Drillz The Rapper - You Want[Remix] feat. King Ten 17
	 ZED FM 17
	 Balance ft Karole Kasita 12
	 Empathy Remix feat APass x Ledra 11
	 LOVE LOAN 10
	 NDONGO 10
Cluster #4
	 All i need feat High Priest 75
	 Tinder 57
	 Koyo Ngic 57
	 Commando ft Tpaul 31
	 Omutima waghi 25
	 BLACK MONDAY 23
	 Gi

In [ ]:
# Get User Cluster and show user name, user rates, user_id

##User cluster
trained_user_embeddings = model.user_factors.weight.data.cpu().numpy()
len(trained_user_embeddings)  # unique song factor weights
trained_user_embeddings


#fit the clusters based on the user weights
user_kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_user_embeddings)


for cluster in range(10):
  print("Cluster #{}". format(cluster))
  users = []
  for useridx in np.where(user_kmeans.labels_ == cluster)[0]:
    userId = train_set.idx2userid[useridx]
    rat_count = ratings_df.loc[ratings_df['user_id']==userId].count()[0]
    users.append((user_names[userId], rat_count,userId))
    # print("song {}--{}".format(song_names[trackId],rat_count))

  for user in sorted(users, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t {} - (rating_total: {}) - ID: {}".format(user[0],user[1], user[2]))

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster #0
	 Sedrick Otolo - (rating_total: 154) - ID: mwZy28HEiwVDWh6zMvii8iY0F5dDz1
	 Payizus - (rating_total: 55) - ID: mw63a2ff5f92834Bon
	 Olweny Joshua - (rating_total: 35) - ID: mw63acf228c2908Jos
	 ogwang - (rating_total: 31) - ID: mw63e13e2897747too
	 Sedu Otolo - (rating_total: 30) - ID: mwuWKfJpyGPrZAbgXCDlKBzDG5cPN2
	 Acar - (rating_total: 25) - ID: mw608bea21b74cbMan
	 Nam Ronny - (rating_total: 22) - ID: mwTzF5gyT0CZhvIg09Ds69777xP7Z2
	 Nalubega Rose - (rating_total: 21) - ID: mw63e49a4e33f84Lan
	 joerolem2021 - (rating_total: 19) - ID: mw63df60cc88a5ejoe
	 KORYANG GODFREY OGWANG - (rating_total: 18) - ID: mw63c15b746a771ogw
Cluster #1
	 Kinyera ivan ouma - (rating_total: 79) - ID: mw6375a9ee98402kin
	 Moonboy - (rating_total: 79) - ID: mw616dd976bb1b1Zap
	 mugume - (rating_total: 7) - ID: mw63d3a969465abwyc
	 Morris - (rating_total: 6) - ID: mw636258041e8f6oke
	 Trevor Trend - (rating_total: 3) - ID: mw63b749c92ee1fTr?
	 Toolit - (rating_total: 2) - ID: mw62b5ffcba7087Iv